<a href="https://colab.research.google.com/github/JonathanJuez/Analisis_sentimientos/blob/main/Sentiment_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#  Importar dependencias


In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup #codificacion de metadatos

from google.colab import drive

In [2]:
try: #control de ultima version TF
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds 

# Prepocesado

## Carga Dataset



In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
train_data = pd.read_csv(
    "/content/drive/My Drive/Datasets/training.1600000.processed.noemoticon.csv",
    header=None, #dataset no contiene cabecera
    names=cols,
    engine="python", #motor de carga
    encoding="latin1"
)
test_data = pd.read_csv(
    "/content/drive/My Drive/Datasets/testdata.manual.2009.06.14.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [5]:
train_data.head(10)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [6]:
data = train_data

## Preprocesamiento

### Data cleaning


In [7]:
data.drop(["id", "date", "query", "user"],
          axis=1,# delete columns
          inplace=True) 

In [8]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Eliminamos los links de las URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Nos quedamos solamente con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]


In [10]:
set(data.sentiment)

{0, 4}

In [11]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [12]:
data.tail(5)

,sentiment,text
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,happy #charitytuesday @theNSPCC @SparksCharity...


In [13]:
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

### Tokenizacion 

In [14]:
#tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(data_clean, targed_vocab_size = 2**13)

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
           data_clean, target_vocab_size=2**16)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [15]:
data_inputs[0]

[65316,
 1570,
 113,
 65323,
 10,
 6,
 3553,
 1,
 135,
 5262,
 50,
 1484,
 38165,
 16,
 13337,
 606,
 2,
 49,
 33,
 1,
 65352]

### Padding

In [16]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [17]:
data_inputs

array([[65316,  1570,   113, ...,     0,     0,     0],
       [   11,  1090,    23, ...,     0,     0,     0],
       [65316,     3, 41563, ...,     0,     0,     0],
       ...,
       [  927,    12,   229, ...,     0,     0,     0],
       [  366,   337,  1309, ...,     0,     0,     0],
       [  181, 51236,     0, ...,     0,     0,     0]], dtype=int32)

### Split Train / test set

In [18]:
test_idx = np.random.randint(0, 800000, 8000) #tweets negativos
test_idx = np.concatenate((test_idx, test_idx+800000)) #tweets positivos

In [19]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

In [20]:
test_inputs.shape

(16000, 73)

In [21]:
test_labels.shape

(16000,)

In [22]:
train_inputs.shape

(1584106, 73)

In [23]:
train_labels.shape

(1584106,)

# Train model

In [25]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [26]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

# Funcion modelo

In [28]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Configuración

In [27]:
VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2#len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

# Entrenamiento

In [29]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [30]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

## Flags

In [38]:
checkpoint_path = "./drive/My Drive/Datasets/checkpoint/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

# Evaluación

In [32]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

500/500 [==============================] - 11s 22ms/step - loss: 0.6934 - accuracy: 0.4947
[0.6933709383010864, 0.4946874976158142]


In [36]:
print("Sentimiento por encima de 0.5 es positiva, si es menor a 0.5 es negativa:") 

Dcnn(np.array([tokenizer.encode("I am been loving you")]), training=False).numpy()

Sentimiento por encima de 0.5 es positiva, si es menor a 0.5 es negativa:


array([[0.4998808]], dtype=float32)

In [37]:
tokenizer.encode("You are so funny")

[135, 39, 21, 1034]